In [39]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import random

In [40]:
coupon_list_train = pd.read_csv('../input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('../input/coupon_list_test.csv')
user_list = pd.read_csv('../input/user_list.csv')
coupon_purchases_train = pd.read_csv("../input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('../input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('../input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('../input/coupon_visit_train.csv')

submession = pd.read_csv('../input/sample_submission.csv')

In [41]:
coupon_purchases_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:54,銀座・新橋・東京・上野,1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:49,恵比寿・目黒・品川,36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:53,恵比寿・目黒・品川,2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:52,恵比寿・目黒・品川,4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
5,1,2011-07-16 00:57:09,恵比寿・目黒・品川,d8b030c8a4a2c1051997092a2985b373,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
6,2,2011-07-16 00:58:29,恵比寿・目黒・品川,2c98138766edf5d5be97ed96aa188c7d,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
7,2,2011-09-07 00:04:47,渋谷・青山・自由が丘,5a45d62c72a4cc4155786b03bec74c96,560574a339f1b25e57b0221e486907ed,259a22533cab381464c89f616a56d12d
8,1,2011-10-19 22:21:50,渋谷・青山・自由が丘,592bb67c0553648d6ebd99a4442606f8,560574a339f1b25e57b0221e486907ed,f4a6f861a266799b07cd99c26e80962b
9,1,2011-12-01 16:53:45,恵比寿・目黒・品川,509883c481716b1f3abeac8372b36a5d,560574a339f1b25e57b0221e486907ed,4257a8a169b40bbbcb002ec47ab3de13


In [37]:
submession

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,NaN
1,00035b86e6884589ec8d28fbf2fe7757,NaN
2,0005b1068d5f2b8f2a7c978fcfe1ca06,NaN
3,000cc06982785a19e2a2fdb40b1c9d59,NaN
4,0013518e41c416cd6a181d277dd8ca0b,NaN
5,001acdee812a18acfd7509172bed5700,NaN
6,001fd7876e3aa29393537c6baf308e43,NaN
7,002383753c1e5d6305c8aff6f89e26d6,NaN
8,0025cae7997d25ea5cf8851bb099c798,NaN
9,002822059a01d895fad84f2f2ff5c1f1,NaN


In [35]:
coupon_purchases_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:54,銀座・新橋・東京・上野,1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:49,恵比寿・目黒・品川,36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:53,恵比寿・目黒・品川,2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:52,恵比寿・目黒・品川,4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
5,1,2011-07-16 00:57:09,恵比寿・目黒・品川,d8b030c8a4a2c1051997092a2985b373,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
6,2,2011-07-16 00:58:29,恵比寿・目黒・品川,2c98138766edf5d5be97ed96aa188c7d,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
7,2,2011-09-07 00:04:47,渋谷・青山・自由が丘,5a45d62c72a4cc4155786b03bec74c96,560574a339f1b25e57b0221e486907ed,259a22533cab381464c89f616a56d12d
8,1,2011-10-19 22:21:50,渋谷・青山・自由が丘,592bb67c0553648d6ebd99a4442606f8,560574a339f1b25e57b0221e486907ed,f4a6f861a266799b07cd99c26e80962b
9,1,2011-12-01 16:53:45,恵比寿・目黒・品川,509883c481716b1f3abeac8372b36a5d,560574a339f1b25e57b0221e486907ed,4257a8a169b40bbbcb002ec47ab3de13


In [22]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash', how='inner')

In [23]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [24]:
purchased_coupons_train = purchased_coupons_train[features]

In [26]:
# 100 ROWS
df = purchased_coupons_train.copy()
df = df[:100] 

In [28]:
df['DISCOUNT_PRICE'] = 1 / np.log10(df['DISCOUNT_PRICE'])
df['PRICE_RATE'] = (df['PRICE_RATE'] / 100) ** 2

In [29]:
df_temp = df.copy()

categorical_cols = ['USER_ID_hash', 'GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name']

In [30]:
df_temp

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,34c48f84026e08355dc3bd19b427f09a,dd8a79645e8170db6a91bad1d89b4055,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,34c48f84026e08355dc3bd19b427f09a,658249379aaaf2b2320061e5218c0c2c,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,34c48f84026e08355dc3bd19b427f09a,1c4aad19f033d97dc163b481116de322,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,34c48f84026e08355dc3bd19b427f09a,e78ae0887c4a9dce91ff0d2a45a9f72c,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,34c48f84026e08355dc3bd19b427f09a,39cabe60f73ea8cad96d7f895ea944ee,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,34c48f84026e08355dc3bd19b427f09a,fca1e37548c9ca5415a9850a1b9b2c28,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,34c48f84026e08355dc3bd19b427f09a,3511c63ae9ce45fc3b5cf5054956d5a4,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,34c48f84026e08355dc3bd19b427f09a,77b92939627690028e22834d7920bf4e,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,34c48f84026e08355dc3bd19b427f09a,1ce49ceb6f55d3b3ed3180f811ec3ffb,宅配,0.312766,0.6084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [26]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: le.fit_transform(col))
df_temp = df_temp.fillna(-1)

In [27]:
df_temp

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,34c48f84026e08355dc3bd19b427f09a,77,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
1,34c48f84026e08355dc3bd19b427f09a,80,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
2,34c48f84026e08355dc3bd19b427f09a,34,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
3,34c48f84026e08355dc3bd19b427f09a,5,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
4,34c48f84026e08355dc3bd19b427f09a,85,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
5,34c48f84026e08355dc3bd19b427f09a,16,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
6,34c48f84026e08355dc3bd19b427f09a,94,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
7,34c48f84026e08355dc3bd19b427f09a,13,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
8,34c48f84026e08355dc3bd19b427f09a,42,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
9,34c48f84026e08355dc3bd19b427f09a,7,0,0.312766,0.6084,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0


In [15]:
l = df_temp.shape[0]
f = len(df_temp.columns)
t = AnnoyIndex(f, 'angular')

In [16]:
for i in range(l):
    v = [df_temp.vsubmessionalues[i][z] for z in range(f)]
    t.add_item(i, v)

TypeError: a float is required

In [ ]:
t.build(50)

In [18]:
sim = t.get_nns_by_item(105, 10)

IndexError: Item index larger than the largest item index

In [24]:
df_temp.loc[sim].drop_duplicates('USER_ID_hash')

,COUPON_ID_hash,USER_ID_hash,GENRE_NAME,DISCOUNT_PRICE,PRICE_RATE,USABLE_DATE_MON,USABLE_DATE_TUE,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
105,17,2602,3,0.453696,0.2500,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,8
3223,19,2972,3,0.393071,0.2809,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,4,9,7
122,17,2962,3,0.453696,0.2500,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,7,8
3222,19,2652,3,0.393071,0.2809,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,4,9,7
3237,19,3205,3,0.393071,0.2809,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,4,9,7


In [17]:
np.mean(df_temp.loc[sim].drop_duplicates('USER_ID_hash').to_numpy(), axis=0)

array([1.82000000e+01, 2.87860000e+03, 3.00000000e+00, 4.17321072e-01,
       2.68540000e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       4.00000000e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 4.00000000e+00, 8.20000000e+00,
       7.40000000e+00])